In [3]:
from ete3 import NCBITaxa
import pandas as pd
import os
import numpy as np
ncbi = NCBITaxa()

NCBI database not present yet (first time used?)
Done. Parsing...


Loading node names...
1548246 names loaded.
183362 synonyms loaded.
Loading nodes...
1548246 nodes loaded.
Linking nodes...
Tree is loaded.
Updating database: /home/gamran/.etetoolkit/taxa.sqlite ...
 1548000 generating entries... 

Inserting synonyms:      20000 


Uploading to /home/gamran/.etetoolkit/taxa.sqlite



Inserting taxid merges:  40000 

Inserting taxids:       10000 

Inserting taxids:       1545000 

In [ ]:
#-----------------------------------------------------------------------

In [1265]:
# MOVE DOWN TO BOTTOM OF PAGE
#Filter the noblasthit fasta file to include reads not hitting the NCBI database, export to new fasta file
noblasthit_input_dir = '/home/gamran/data/noblasthit'
noblasthit_files_in_folder = sorted([x for x in list(os.walk(noblasthit_input_dir, topdown=False))][-1][2])
noblasthit_bcs = sorted(list(NCBI_dfs_dict.keys()))

from Bio import SeqIO
os.chdir('/home/gamran/data/noblasthit')
outdir = '/home/gamran/analysis/'
os.curdir

for ind in range(0, len(noblasthit_bcs)):
    noNCBIhit_id = ''
    noNCBIhit_id = noblasthit_files_in_folder[ind][:-3] + ".WW_10012017.noncbihit.fa"
    not_NCBIhit_sequences = []
    for seq in SeqIO.parse(open(noblasthit_files_in_folder[ind]), 'fasta'):
        if seq.id not in list(NCBI_dfs_dict[noblasthit_bcs[ind]]['qseqid']):
            not_NCBIhit_sequences.append(seq)
    print(noblasthit_bcs[ind], len(not_NCBIhit_sequences))
    SeqIO.write(not_NCBIhit_sequences, noNCBIhit_id, "fasta")
    not_NCBIhit_sequences = []


BC00 184
BC01 10029
BC02 8386
BC03 1543
BC04 3541
BC05 3435
BC06 26083
NB00 5359


In [ ]:
#-----------------------------------------------------------------------

In [4]:
#Constructing dataframes for each individual blasted barcode

os.chdir('/home/gamran/blast/')
outdir = '/home/gamran/analysis/'
os.curdir

'.'

In [18]:
blast_input_dir = '/home/gamran/blast'
blast_files_in_folder = [x for x in list(os.walk(blast_input_dir, topdown=False))][1][2]
NCBI_files_in_folder = sorted([x for x in blast_files_in_folder if 'NCBI' in x])

['1212_1D_barcoding_Wagga_BC00.WW_19121016.noblasthit.fa.NCBI04052016.001.besthit',
 '1212_1D_barcoding_Wagga_BC00.WW_19121016.noblasthit.fa.NCBI04052016.besthit',
 '1212_1D_barcoding_Wagga_BC01.WW_19121016.noblasthit.fa.NCBI04052016.001.besthit',
 '1212_1D_barcoding_Wagga_BC01.WW_19121016.noblasthit.fa.NCBI04052016.besthit',
 '1212_1D_barcoding_Wagga_BC02.WW_19121016.noblasthit.fa.NCBI04052016.001.besthit',
 '1212_1D_barcoding_Wagga_BC02.WW_19121016.noblasthit.fa.NCBI04052016.besthit',
 '1212_1D_barcoding_Wagga_BC03.WW_19121016.noblasthit.fa.NCBI04052016.001.besthit',
 '1212_1D_barcoding_Wagga_BC03.WW_19121016.noblasthit.fa.NCBI04052016.besthit',
 '1212_1D_barcoding_Wagga_BC04.WW_19121016.noblasthit.fa.NCBI04052016.besthit',
 '1212_1D_barcoding_Wagga_BC05.WW_19121016.noblasthit.fa.NCBI04052016.001.besthit',
 '1212_1D_barcoding_Wagga_BC05.WW_19121016.noblasthit.fa.NCBI04052016.besthit',
 '1212_1D_barcoding_Wagga_BC06.WW_19121016.noblasthit.fa.NCBI04052016.001.besthit',
 '1212_1D_barcod

In [30]:
#Creating a dictionary for NCBI blasted barcodes - key: 'barcode', value: related NCBI blast dataframe
NCBI_dfs_dict = {}
for file_name in NCBI_files_in_folder:
    if '.001.' in file_name:
        continue
    elif '.besthit' in file_name:
        key = '' #Safety 
        key = file_name.split('.')[0][-4:] # Try to make a pattern match, not an index hard code
        NCBI_dfs_dict[key] = pd.read_table(file_name, delim_whitespace=False, 
                                           names=['qseqid', 'sseqid', 'evalue', 'bitscore', 'length', 
                                                  'pident', 'nident', 'sgi', 'sacc', 'staxids', 'sscinames', 
                                                  'scomnames', 'sskingdoms', 'barcode']) #extra barcode column added
    continue

In [32]:
#Assigning barcodes to the barcode column based on dict key
for bcs, dfs in NCBI_dfs_dict.items():
    for i in range(0, len(dfs)):
        dfs.loc[i, 'barcode'] = bcs

In [37]:
#Joining the dataframes for individual barcodes together
NCBI_df_list = []
for bcs in NCBI_dfs_dict.keys():
    NCBI_df_list.append(NCBI_dfs_dict[bcs])
    
joint_NCBI_df = pd.concat(NCBI_df_list, axis=0, ignore_index=True)
joint_NCBI_df.sort_values('barcode', ascending=True, inplace=True)
joint_NCBI_df.reset_index(drop=True, inplace=True)
#joint_NCBI_df

In [1264]:
print(len(NCBI_dfs_dict['BC01']))
print(len(joint_NCBI_df.loc[joint_NCBI_df['barcode'] == 'BC01']))
#joint_NCBI_df.tail()

451
451


In [260]:
#Create pivot table showing counts of species (names) per barcode, sorted by control (BC04)
joint_NCBI_pivot_table_names = joint_NCBI_df.pivot_table(values='nident', index='scomnames', columns='barcode', aggfunc='count', fill_value=0, margins=True)
joint_NCBI_pivot_table_names.index.name = None
joint_NCBI_pivot_table_names.columns.name = None
joint_NCBI_pivot_table_names.sort_values('BC04', axis=0, ascending=False).head(n=10)


,BC00,BC01,BC02,BC03,BC04,BC05,BC06,NB00,All
All,134.0,451.0,4127.0,1764.0,8746.0,4424.0,3212.0,47.0,22905.0
Cloning vector lambda TXF97,56.0,239.0,1582.0,662.0,3724.0,1282.0,1679.0,16.0,9240.0
Shigella sp. PAMC 28760,45.0,122.0,1311.0,672.0,3672.0,1122.0,1049.0,15.0,8008.0
Enterobacteria phage lambda,7.0,43.0,199.0,89.0,450.0,146.0,179.0,0.0,1113.0
Escherichia coli,4.0,12.0,78.0,27.0,172.0,67.0,52.0,4.0,416.0
Shigella flexneri 1a,2.0,5.0,64.0,29.0,137.0,62.0,34.0,1.0,334.0
Enterobacteria phage HK629,1.0,3.0,51.0,19.0,130.0,62.0,64.0,0.0,330.0
Salmonella enterica subsp. enterica serovar Typhimurium,0.0,5.0,29.0,11.0,80.0,16.0,8.0,1.0,150.0
Escherichia coli P12b,0.0,1.0,7.0,5.0,30.0,6.0,6.0,1.0,56.0
African malaria mosquito,0.0,0.0,13.0,1.0,22.0,8.0,1.0,0.0,45.0


In [380]:
#Create pivot table showing % of species per barcode, sorted by control (BC04)
joint_NCBI_pivot_table_names_pc = joint_NCBI_pivot_table_names.copy()

for bcs in joint_NCBI_pivot_table_names.columns:
    joint_NCBI_pivot_table_names_pc[str(bcs) + '%'] = joint_NCBI_pivot_table_names_pc[bcs].apply(lambda x: (x / joint_NCBI_pivot_table_names[bcs].loc['All'])*100)
for spcs in joint_NCBI_pivot_table_names.columns:
    joint_NCBI_pivot_table_names_pc.drop(spcs, axis=1, inplace=True)

joint_NCBI_pivot_table_names_pc2 = joint_NCBI_pivot_table_names_pc.sort_values('BC04%', axis=0, ascending=False)
joint_NCBI_pivot_table_names_pc2.applymap(lambda x: '%.3f' % x).head(n=15)


,BC00%,BC01%,BC02%,BC03%,BC04%,BC05%,BC06%,NB00%,All%
All,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000
Cloning vector lambda TXF97,41.791,52.993,38.333,37.528,42.579,28.978,52.273,34.043,40.341
Shigella sp. PAMC 28760,33.582,27.051,31.766,38.095,41.985,25.362,32.659,31.915,34.962
Enterobacteria phage lambda,5.224,9.534,4.822,5.045,5.145,3.300,5.573,0.000,4.859
Escherichia coli,2.985,2.661,1.890,1.531,1.967,1.514,1.619,8.511,1.816
Shigella flexneri 1a,1.493,1.109,1.551,1.644,1.566,1.401,1.059,2.128,1.458
Enterobacteria phage HK629,0.746,0.665,1.236,1.077,1.486,1.401,1.993,0.000,1.441
Salmonella enterica subsp. enterica serovar Typhimurium,0.000,1.109,0.703,0.624,0.915,0.362,0.249,2.128,0.655
Escherichia coli P12b,0.000,0.222,0.170,0.283,0.343,0.136,0.187,2.128,0.244
African malaria mosquito,0.000,0.000,0.315,0.057,0.252,0.181,0.031,0.000,0.196


In [420]:
joint_NCBI_df

,qseqid,sseqid,evalue,bitscore,length,pident,nident,sgi,sacc,staxids,sscinames,scomnames,sskingdoms,barcode
0,b694e6b0-be3f-41d4-a1ac-960beaa1e3cd_Basecall_...,gi|1008889473|gb|CP014768.1|,0.000000e+00,1618.0,1512,87.37,1321,1008889473,CP014768,1813821,Shigella sp. PAMC 28760,Shigella sp. PAMC 28760,Bacteria,BC00
1,3e9eaf56-497e-4892-b9e2-b28700044e69_Basecall_...,gi|1051181|gb|U37692.1|CVU37692,0.000000e+00,1792.0,2255,83.15,1875,1051181,U37692,43300,Cloning vector lambda TXF97,Cloning vector lambda TXF97,NaN,BC00
2,3fc3a37e-0151-4ec6-95c9-312e779bd514_Basecall_...,gi|1051181|gb|U37692.1|CVU37692,0.000000e+00,1195.0,1155,86.84,1003,1051181,U37692,43300,Cloning vector lambda TXF97,Cloning vector lambda TXF97,NaN,BC00
3,400652b7-6dff-49de-bf65-1847b08fbffb_Basecall_...,gi|215104|gb|J02459.1|LAMCG,0.000000e+00,3369.0,2587,91.15,2358,215104,J02459,10710,Enterobacteria phage lambda,Enterobacteria phage lambda,Viruses,BC00
4,41c35e20-2205-405b-adab-75132600e591_Basecall_...,gi|1051181|gb|U37692.1|CVU37692,0.000000e+00,1297.0,1373,85.14,1169,1051181,U37692,43300,Cloning vector lambda TXF97,Cloning vector lambda TXF97,NaN,BC00
5,4248f4fe-34e0-4ce5-ade8-d05002a3058d_Basecall_...,gi|445198867|gb|CP004045.1|,0.000000e+00,1327.0,1402,85.24,1195,445198867,CP004045,1282356,Pseudomonas poae RE*1-1-14,Pseudomonas poae RE*1-1-14,Bacteria,BC00
6,4616a9eb-c856-4803-9e74-de9c48ca4486_Basecall_...,gi|1008889473|gb|CP014768.1|,0.000000e+00,1044.0,896,89.06,798,1008889473,CP014768,1813821,Shigella sp. PAMC 28760,Shigella sp. PAMC 28760,Bacteria,BC00
7,4902fe2e-72ea-4eb4-ad5c-51dd2d7d1e67_Basecall_...,gi|1051181|gb|U37692.1|CVU37692,0.000000e+00,1537.0,1757,84.18,1479,1051181,U37692,43300,Cloning vector lambda TXF97,Cloning vector lambda TXF97,NaN,BC00
8,492645b7-63c1-42ca-a901-ebe6a6643870_Basecall_...,gi|1051181|gb|U37692.1|CVU37692,0.000000e+00,743.0,1529,77.50,1185,1051181,U37692,43300,Cloning vector lambda TXF97,Cloning vector lambda TXF97,NaN,BC00
9,49fb902b-070e-4e76-b158-f15952ed933e_Basecall_...,gi|1002167280|gb|CP012733.1|,4.000000e-56,230.0,210,87.62,184,1002167280,CP012733,374923,Shigella flexneri 1a,Shigella flexneri 1a,Bacteria,BC00


In [474]:
#Number of unique tax ids per barcode in the joint_NCBI_pivot_table
tax_id_barcode_len_list = []
for bcs in joint_NCBI_pivot_table_names.columns:
    print(bcs, ' = ', len(joint_NCBI_df.loc[joint_NCBI_df['barcode'] == bcs].staxids.unique()))
    tax_id_barcode_len_list.append(len(joint_NCBI_df.loc[joint_NCBI_df['barcode'] == bcs].staxids.unique()))

#Sum of unique tax ids per barcode in the joint_NCBI_pivot_table
print('All - Unique Per Barcode = ', sum(tax_id_barcode_len_list))
    
#No of unique tax ids across entire joint_NCBI_pivot_table
print('All - Unique Across Dataframe = ', len(list(set(joint_NCBI_df.staxids))))



BC00  =  14
BC01  =  23
BC02  =  151
BC03  =  124
BC04  =  115
BC05  =  270
BC06  =  81
NB00  =  14
All  =  0
All - Unique Per Barcode =  792
All - Unique Across Dataframe =  560


In [429]:
#Generates a dictionary assigning key: barcode, value: list of unique tax ids for each barcode
bcs_staxids_dict = {}
for bcs in joint_NCBI_pivot_table_names.columns:
    bcs_staxids_dict[bcs] = list(joint_NCBI_df.loc[joint_NCBI_df['barcode'] == bcs].staxids.unique())
del bcs_staxids_dict['All']
#bcs_staxids_dict.keys()

['43300',
 '1813821',
 '1524881',
 '31964',
 '7165',
 '10710',
 '1324932',
 '1147148',
 '199201',
 '1357279',
 '374923',
 '294',
 '587753',
 '562',
 '1147146',
 '910348',
 '9606',
 '405955',
 '38735',
 '357240',
 '37554',
 '1147786',
 '749889',
 '1282356',
 '1785145',
 '1332075',
 '634500',
 '574521',
 '1484157',
 '1028989',
 '54291',
 '227321',
 '4555',
 '264483',
 '595496',
 '568708',
 '665912',
 '930166',
 '1047171',
 '37762',
 '101618',
 '1114970',
 '1609977',
 '443906',
 '33014',
 '90371',
 '157739',
 '1287680',
 '634499',
 '1097668',
 '671987',
 '432200',
 '205918',
 '465817',
 '317',
 '321',
 '509925',
 '198628',
 '225338',
 '10761',
 '215689',
 '41514',
 '1649877',
 '264730',
 '1274814',
 '199310',
 '480178',
 '10090',
 '539221',
 '906668',
 '1441629',
 '1324931',
 '208963',
 '716540',
 '223283',
 '573235',
 '406716',
 '4565',
 '85698',
 '1692238',
 '1147145',
 '861557',
 '28447',
 '157783',
 '1028729',
 '889211',
 '578455',
 '225343',
 '1415630',
 '1221522',
 '37682',
 '149533

In [556]:
#Determining which barcodes have unique tax id lists with multiples
def find_mult_staxids(bcs):
    """Function to determine whether a list of unique tax ids contains a multiple e.g. taxid1;taxid2"""
    for staxids_index in range(0, len(bcs_staxids_dict[bcs])):
        if ';' in str(bcs_staxids_dict[bcs][staxids_index]):
            print(bcs_staxids_dict[bcs][staxids_index])
            print(bcs_staxids_dict[bcs].index(bcs_staxids_dict[bcs][staxids_index]))
            print(bcs)
            print()
            
for bcs in sorted(list(bcs_staxids_dict.keys())):
    find_mult_staxids(bcs)

225338;985895
100
BC02

5266;5298;5301
107
BC04

225338;985895
84
BC05

97479;861557
192
BC05



In [608]:
#Generates a dictionary assigning key: barcode, value: lineage corresponding to each unique tax id for each barcode
def generate_bcs_lineage_list(bcs):
    """Generates a value (lineage list for staxids in a specific barcode), 
       assigns to key (barcode) in a dictionary (bcs_lineage_dict)"""
    lineage_list = []
    for staxids in list(joint_NCBI_df.loc[joint_NCBI_df['barcode'] == bcs].staxids.unique()):
        if ';' in str(staxids):
            staxids = staxids.split(';')[0] #for taxids in multiples, splits and only takes first one
        lineage = ''
        lineage = ncbi.get_lineage(staxids)
        lineage_list.append(list(lineage))
    bcs_lineage_dict[bcs] = lineage_list
    
bcs_lineage_dict = {}
for bcs in joint_NCBI_pivot_table_names.columns:
    generate_bcs_lineage_list(bcs)
del bcs_lineage_dict['All']

In [613]:
#Generates a dictionary assigning key: barcode, value: lineage names corresponding to each unique tax id for each barcode
def generate_bcs_lineage_names_list(bcs):
    """Generates a value (lineage names list for staxids in a specific barcode), 
       assigns to key (barcode) in a dictionary (bcs_lineage_names_dict)"""
    lineage_names_list = []
    for staxids in list(joint_NCBI_df.loc[joint_NCBI_df['barcode'] == bcs].staxids.unique()):
        if ';' in str(staxids):
            staxids = staxids.split(';')[0]
        lineage = ''
        lineage = ncbi.get_lineage(staxids)
        names = ''
        names = ncbi.get_taxid_translator(lineage)
        names_taxid = [names[taxid] for taxid in lineage]
        lineage_names_list.append(list(names_taxid))
    bcs_lineage_names_dict[bcs] = lineage_names_list
    
bcs_lineage_names_dict = {}
for bcs in joint_NCBI_pivot_table_names.columns:
    generate_bcs_lineage_names_list(bcs)
del bcs_lineage_names_dict['All']

In [724]:
def generate_bcs_lineage_ranks_list(bcs):
    """Generates a value (lineage ranks list for staxids in a specific barcode), 
       assigns to key (barcode) in a dictionary (bcs_lineage_names_dict)"""
    lineage_ranks_list = []
    for staxids in list(joint_NCBI_df.loc[joint_NCBI_df['barcode'] == bcs].staxids.unique()):
        if ';' in str(staxids):
            staxids = staxids.split(';')[0]
        lineage = ''
        lineage = ncbi.get_lineage(staxids)
        ranks = ''
        ranks = ncbi.get_rank(lineage)
        ranks_taxid = [ranks[taxid] for taxid in lineage]
        lineage_ranks_list.append(list(ranks_taxid))
    bcs_lineage_ranks_dict[bcs] = lineage_ranks_list

bcs_lineage_ranks_dict = {}
for bcs in joint_NCBI_pivot_table_names.columns:
    generate_bcs_lineage_ranks_list(bcs)
del bcs_lineage_ranks_dict['All']

In [1074]:
#Generates a dictionary assigning key: barcode, value: list [lineage names, taxonomic ranks] corresponding to each unique tax id for each barcode
def generate_bcs_lineage_names2ranks_list(bcs):
    """Generates a value ( [lineage names, ranks] list for staxids in a specific barcode), 
       assigns to key (barcode) in a dictionary (bcs_lineage_names_dict)"""    
    lineage_names2ranks_list = []
    for lineage_index in range(0, len(bcs_lineage_names_dict[bcs])):
        names = ''
        names = bcs_lineage_names_dict[bcs][lineage_index]
        ranks = ''
        ranks = bcs_lineage_ranks_dict[bcs][lineage_index]
        names2ranks = [tuple(x) for x in zip(names, ranks)]
        lineage_names2ranks_list.append(names2ranks)
    bcs_lineage_names2ranks_dict[bcs] = lineage_names2ranks_list
    
bcs_lineage_names2ranks_dict = {}
for bcs in joint_NCBI_pivot_table_names.columns:
    generate_bcs_lineage_names2ranks_list(bcs)
    

In [872]:
#Adds a correct 'All' categories to the above dictionaries
lineage_list = []
lineage_names_list = []
lineage_ranks_list = []
unique_staxids = joint_NCBI_df['staxids'].unique()

for staxids in unique_staxids:
    if ';' in str(staxids):
            staxids = staxids.split(';')[0]
    lineage = ''
    lineage = ncbi.get_lineage(staxids)
    lineage_list.append(list(lineage))
    
    names = ''
    names = ncbi.get_taxid_translator(lineage)
    names_taxid = [names[taxid] for taxid in lineage]
    lineage_names_list.append(list(names_taxid))
    
    ranks = ncbi.get_rank(lineage)
    ranks_taxid = [ranks[taxid] for taxid in lineage]
    lineage_ranks_list.append(list(ranks_taxid))

#Why are some in quotes (strings) and some are not???
bcs_staxids_dict['All'] = list(joint_NCBI_df['staxids'].unique()) #Note: 'All' contains the taxid multiples, the others don't
bcs_lineage_dict['All'] = lineage_list
bcs_lineage_names_dict['All'] = lineage_names_list
bcs_lineage_ranks_dict['All'] = lineage_ranks_list

In [979]:
bcs_lineage_ranks_dict['All']

[['no rank',
  'no rank',
  'superkingdom',
  'phylum',
  'class',
  'order',
  'family',
  'genus',
  'species'],
 ['no rank', 'no rank', 'no rank', 'no rank', 'species'],
 ['no rank', 'superkingdom', 'no rank', 'order', 'family', 'genus', 'species'],
 ['no rank',
  'no rank',
  'superkingdom',
  'phylum',
  'class',
  'order',
  'family',
  'genus',
  'species group',
  'species',
  'no rank'],
 ['no rank',
  'no rank',
  'superkingdom',
  'phylum',
  'class',
  'order',
  'family',
  'genus',
  'species',
  'no rank'],
 ['no rank',
  'no rank',
  'superkingdom',
  'phylum',
  'class',
  'order',
  'family',
  'genus',
  'species group',
  'species subgroup',
  'species',
  'no rank',
  'no rank'],
 ['no rank',
  'no rank',
  'superkingdom',
  'no rank',
  'kingdom',
  'subkingdom',
  'phylum',
  'no rank',
  'subphylum',
  'no rank',
  'no rank',
  'class',
  'subclass',
  'order',
  'family',
  'genus',
  'species'],
 ['no rank',
  'no rank',
  'superkingdom',
  'phylum',
  'class'

In [1079]:
bcs_lineage_names2ranks_dict['BC00'][0]

[('root', 'no rank'),
 ('cellular organisms', 'no rank'),
 ('Bacteria', 'superkingdom'),
 ('Proteobacteria', 'phylum'),
 ('Gammaproteobacteria', 'class'),
 ('Enterobacterales', 'order'),
 ('Enterobacteriaceae', 'family'),
 ('Shigella', 'genus'),
 ('Shigella sp. PAMC 28760', 'species')]

In [916]:
#Checks for code working, all should be the same length
#print(len(bcs_staxids_dict['BC05']))
#print(len(bcs_lineage_dict['BC05']))
#print(len(bcs_lineage_names_dict['BC05']))
#print(len(bcs_lineage_ranks_dict['BC05']))
#print(len(bcs_lineage_names2ranks_dict['BC05']))

In [1151]:
#Trying to find discern all the available taxonomic ranks used across the NCBI dataframe
unique_rank_list = []
for unique_id in range(0, len(bcs_lineage_names2ranks_dict['All'])):
    for lineages in bcs_lineage_names2ranks_dict['All'][unique_id]:
        unique_rank_list.append(lineages[1])
print(sorted(list(set(unique_rank_list))))

['class', 'family', 'forma', 'genus', 'infraclass', 'infraorder', 'kingdom', 'no rank', 'order', 'parvorder', 'phylum', 'species', 'species group', 'species subgroup', 'subclass', 'subfamily', 'subgenus', 'subkingdom', 'suborder', 'subphylum', 'subspecies', 'subtribe', 'superclass', 'superfamily', 'superkingdom', 'superorder', 'tribe', 'varietas']


In [1185]:
#Determining what unique staxids contain or do not contain a certain taxonomic rank
def check_rank_in_taxids(rank_name, dict_w_key, rank_present=True):
    """Given a dictionary value listing lineages (dict_w_key) and a specific taxonomic rank
       to search (rank_name), find lineages with or without the specific rank and their index"""
    for unique_id in range(0, len(dict_w_key)):
        if rank_present == True:
            if rank_name in [pairs for lineages in dict_w_key[unique_id] for pairs in lineages]:
                print('Yes!', unique_id, dict_w_key[unique_id])
                print()
            else:
                None
        if rank_present == False:
            if rank_name in [pairs for lineages in dict_w_key[unique_id] for pairs in lineages]:
                None
            else:
                print('No!', unique_id, dict_w_key[unique_id])
                print()

In [1190]:
check_rank_in_taxids('unclassified sequences', bcs_lineage_names2ranks_dict['All'], rank_present=True)

Yes! 483 [('root', 'no rank'), ('unclassified sequences', 'no rank'), ('environmental samples', 'no rank'), ('uncultured microorganism', 'species')]



In [1191]:
#Prints several taxonomic ranks that follow the one specified, none if rank doesn't exist
for lists in bcs_lineage_names_dict['All']:
    if 'Viruses' in lists:
        n1 = 2
        n2 = 3
        n3 = 4
        n4 = 5
        print(lists[-1], '', end="")
        print('| Ranks = ', len(lists))
        print(lists[n1], ', ', end="") if n1 < len(lists) else print('N/A')
        print(lists[n2], ', ', end="") if n2 < len(lists) else print('N/A')
        print(lists[n3], ', ', end="") if n3 < len(lists) else print('N/A')
        print(lists[n4]) if n4 < len(lists) else print('N/A')
        print()

Escherichia virus Lambda | Ranks =  7
dsDNA viruses, no RNA stage , Caudovirales , Siphoviridae , Lambdavirus

Enterobacteria phage HK629 | Ranks =  8
dsDNA viruses, no RNA stage , Caudovirales , Siphoviridae , Lambdavirus

Salmonella phage epsilon34 | Ranks =  8
dsDNA viruses, no RNA stage , Caudovirales , Podoviridae , P22virus

Enterobacteria phage HK542 | Ranks =  7
dsDNA viruses, no RNA stage , Caudovirales , Siphoviridae , unclassified Siphoviridae

Enterobacteria phage CUS-3 | Ranks =  8
dsDNA viruses, no RNA stage , Caudovirales , Podoviridae , P22virus

Shigella phage Sf6 | Ranks =  7
dsDNA viruses, no RNA stage , Caudovirales , Podoviridae , P22virus

Enterobacteria phage Sf101 | Ranks =  7
dsDNA viruses, no RNA stage , Caudovirales , Podoviridae , unclassified Podoviridae

Escherichia virus Lambda | Ranks =  7
dsDNA viruses, no RNA stage , Caudovirales , Siphoviridae , Lambdavirus

Enterobacteria phage HK629 | Ranks =  8
dsDNA viruses, no RNA stage , Caudovirales , Siphoviri

In [1086]:
#If the unique id has a certain set upper rank, print lower rank
def if_upper_print_lower(upr_ind, upr_rnk, lwr_ind, dict_w_key):
    for unique_id in range(0, len([x[upr_ind] for x in dict_w_key])):
        if [x[upr_ind] for x in dict_w_key][unique_id] == upr_rnk:
            print([x[lwr_ind] for x in dict_w_key][unique_id])
        else:
            print('Not ' + upr_rnk)

print(set([x[3] for x in bcs_lineage_names_dict['All']]))            
print()
if_upper_print_lower(2, 'Eukaryota', 3, bcs_lineage_names_dict['All'])

{'FCB group', 'vectors', 'Euglenozoa', 'Viridiplantae', 'uncultured microorganism', 'Stramenopiles', 'Proteobacteria', 'environmental samples', 'Baculoviridae', 'Opisthokonta', 'Caudovirales', 'Amoebozoa', 'Spirochaetes', 'Plasmid ColE1', 'unclassified Bacteria', 'Terrabacteria group', 'Alveolata'}

Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Opisthokonta
Not Eukaryota
Not Eukaryota
Not Eukaryota
Opisthokonta
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Viridiplantae
Opisthokonta
Opisthokonta
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
environmental samples
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Opisthokonta
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Not Eukaryota
Opisthokonta
Not Eukaryota
Viridiplantae
Not Eukaryota
Not Eukaryota
Not Eukaryota
Opisthokont

In [1172]:
set([x[2] for x in bcs_lineage_names2ranks_dict['All']])

{('Bacteria', 'superkingdom'),
 ('Eukaryota', 'superkingdom'),
 ('artificial sequences', 'no rank'),
 ('dsDNA viruses, no RNA stage', 'no rank'),
 ('environmental samples', 'no rank'),
 ('plasmids', 'no rank')}

In [1178]:
#Shows all available species in the NCBi dataframe
species_list = []
for lineages in bcs_lineage_names2ranks_dict['All']:
    for ranks in lineages:
        if 'species' in ranks:
            species_list.append(ranks)
set(species_list)

{('Achromobacter xylosoxidans', 'species'),
 ('Acidosasa purpurea', 'species'),
 ('Actinoplanes sp. SE50/110', 'species'),
 ('Acytostelium subglobosum', 'species'),
 ('Aegilops crassa', 'species'),
 ('Aegilops geniculata', 'species'),
 ('Aegilops speltoides', 'species'),
 ('Aegilops tauschii', 'species'),
 ('Agrobacterium tumefaciens', 'species'),
 ('Albugo laibachii', 'species'),
 ('Alicycliphilus denitrificans', 'species'),
 ('Altererythrobacter epoxidivorans', 'species'),
 ('Alternaria brassicicola', 'species'),
 ('Alternaria infectoria', 'species'),
 ('Anopheles gambiae', 'species'),
 ('Anthoxanthum odoratum', 'species'),
 ('Aristolochia macrophylla', 'species'),
 ('Aspergillus nidulans', 'species'),
 ('Aspergillus niger', 'species'),
 ('Aureobasidium namibiae', 'species'),
 ('Aureobasidium subglaciale', 'species'),
 ('Auricularia subglabra', 'species'),
 ('Azoarcus sp. KH32C', 'species'),
 ('Azospirillum brasilense', 'species'),
 ('BAC cloning vector pEZ BAC', 'species'),
 ('Bacil

In [1266]:
for lineages in bcs_lineage_names2ranks_dict['All'][i]



[[('root', 'no rank'),
  ('cellular organisms', 'no rank'),
  ('Bacteria', 'superkingdom'),
  ('Proteobacteria', 'phylum'),
  ('Gammaproteobacteria', 'class'),
  ('Enterobacterales', 'order'),
  ('Enterobacteriaceae', 'family'),
  ('Shigella', 'genus'),
  ('Shigella sp. PAMC 28760', 'species')],
 [('root', 'no rank'),
  ('other sequences', 'no rank'),
  ('artificial sequences', 'no rank'),
  ('vectors', 'no rank'),
  ('Cloning vector lambda TXF97', 'species')],
 [('root', 'no rank'),
  ('Viruses', 'superkingdom'),
  ('dsDNA viruses, no RNA stage', 'no rank'),
  ('Caudovirales', 'order'),
  ('Siphoviridae', 'family'),
  ('Lambdavirus', 'genus'),
  ('Escherichia virus Lambda', 'species')],
 [('root', 'no rank'),
  ('cellular organisms', 'no rank'),
  ('Bacteria', 'superkingdom'),
  ('Proteobacteria', 'phylum'),
  ('Gammaproteobacteria', 'class'),
  ('Pseudomonadales', 'order'),
  ('Pseudomonadaceae', 'family'),
  ('Pseudomonas', 'genus'),
  ('Pseudomonas fluorescens group', 'species grou

BC00    41.791
BC01    52.993
BC02    38.333
BC03    37.528
BC04    42.579
BC05    28.978
BC06    52.273
NB00    34.043
All     40.341
dtype: float64

Index(['Achromobacter xylosoxidans', 'Acidosasa purpurea',
       'Actinoplanes sp. SE50/110', 'Acytostelium subglobosum LB1',
       'Aegilops crassa', 'Aegilops geniculata', 'Aegilops speltoides',
       'Aegilops tauschii', 'African malaria mosquito',
       'Agrobacterium radiobacter K84',
       ...
       'uncultured bacterium HH1107', 'uncultured eukaryote',
       'uncultured fungus', 'uncultured marine bacterium',
       'uncultured microorganism', 'upland cotton', 'wild Malaysian banana',
       'wild oats', 'wine grape', 'All'],
      dtype='object', length=444)

In [ ]:
joint_NCBI_pivot_table_names_pc.drop('All%', axis=1, inplace=True)


In [104]:
#Shows ten most prevalent species per barcode
def ten_best_species_names(bcs):
    """Shows ten most prevalent species for a single barcode"""
    print(joint_NCBI_pivot_table_names[bcs].sort_values(axis=0, ascending=False).head(n=10))
    
for bcs in sorted(list(NCBI_dfs_dict.keys())):
    ten_best_species_names(bcs)
    print()

All                                        134.0
Cloning vector lambda TXF97                 56.0
Shigella sp. PAMC 28760                     45.0
Pseudomonas poae RE*1-1-14                   8.0
Enterobacteria phage lambda                  7.0
Escherichia coli                             4.0
Pseudomonas syringae pv. syringae HS191      4.0
Zymoseptoria tritici                         2.0
Shigella flexneri 1a                         2.0
Bipolaris sorokiniana ND90Pr                 1.0
Name: BC00, dtype: float64

All                                                        451.0
Cloning vector lambda TXF97                                239.0
Shigella sp. PAMC 28760                                    122.0
Enterobacteria phage lambda                                 43.0
Escherichia coli                                            12.0
Shigella flexneri 1a                                         5.0
Salmonella enterica subsp. enterica serovar Typhimurium      5.0
Enterobacteria phage CUS-3 

All                                        134.0
Cloning vector lambda TXF97                 56.0
Shigella sp. PAMC 28760                     45.0
Pseudomonas poae RE*1-1-14                   8.0
Enterobacteria phage lambda                  7.0
Escherichia coli                             4.0
Pseudomonas syringae pv. syringae HS191      4.0
Zymoseptoria tritici                         2.0
Shigella flexneri 1a                         2.0
Bipolaris sorokiniana ND90Pr                 1.0
Name: BC00, dtype: float64

All                                                        451.0
Cloning vector lambda TXF97                                239.0
Shigella sp. PAMC 28760                                    122.0
Enterobacteria phage lambda                                 43.0
Escherichia coli                                            12.0
Shigella flexneri 1a                                         5.0
Salmonella enterica subsp. enterica serovar Typhimurium      5.0
Enterobacteria phage CUS-3 

In [137]:
#No of unique species identified across all barcodes
len(joint_NCBI_df.staxids.unique())

560

In [45]:
NCBI_dfs_dict['NB00'].tail()

,qseqid,sseqid,evalue,bitscore,length,pident,nident,sgi,sacc,staxids,sscinames,scomnames,sskingdoms,barcode
42,e4e15885-c061-4f32-b847-cca6d6858ce3_Basecall_...,gi|1051181|gb|U37692.1|CVU37692,0.000000e+00,1371.0,1111,89.92,999,1051181,U37692,43300,Cloning vector lambda TXF97,Cloning vector lambda TXF97,NaN,NB00
43,eb0b9dc3-d98f-4f96-a45b-e0e3afda927c_Basecall_...,gi|1008889473|gb|CP014768.1|,1.000000e-120,444.0,383,88.77,340,1008889473,CP014768,1813821,Shigella sp. PAMC 28760,Shigella sp. PAMC 28760,Bacteria,NB00
44,ef721feb-2406-4593-9e13-395e3c987f08_Basecall_...,gi|1001869111|gb|CP014522.1|,1.000000e-44,191.0,162,88.27,143,1001869111,CP014522,562,Escherichia coli,Escherichia coli,Bacteria,NB00
45,f7695fba-01fa-4ca7-b6fb-2faa56221862_Basecall_...,gi|1051181|gb|U37692.1|CVU37692,0.000000e+00,1007.0,918,87.91,807,1051181,U37692,43300,Cloning vector lambda TXF97,Cloning vector lambda TXF97,NaN,NB00
46,fb00379a-8abf-46b5-9b1b-5db8ab49e838_Basecall_...,gi|970580326|gb|CP013183.1|,0.000000e+00,1107.0,1087,86.11,936,970580326,CP013183,199201,Pseudomonas syringae pv. lapsa,Pseudomonas syringae pv. lapsa,Bacteria,NB00
